# Capstone Project: The Battle of Neighbourhoods (Week 2) #

_____

## *1. Introduction* ##

### 1.1 A description of the problem ###

Entertainment place recommendation is a challenging task because of the variance in what an individual perceives as comfortable versus another. For example, children prefer play grounds or theme parks, young people like adventure and adult like some uproarious locations like bars or pubs. Additionally, the preference of entertainment varies in different time intervals or period. For example, people tend to visit natural lanscapes in summer rather than go to museums, or in winter much more custumers usually go to BBQ restaurants instead of the hot weather like summer. Entertainment preference also varies from culture to culture. For example, visiting religious locations like churches or temples would be prefered over stadiums in the weekend. A recommendation system would be highly effective if it can deal with as many of such challenges as possible.

### 1.2 Background ###

As we know, London is one of the most attractive locations in the world. Attracting 27 million visitors every year, London becomes the most visited city in Europe. It’s no surprise that London is top of so many people’s travel plans. Today, London is one of the most diverse cities in the world, with a rich history and some of the most outstanding culture on the planet. Of course, London has many entertainment facilities that being the most interests tourists. However, among a thousands of selections of which places that are convenient, high-rated, and affordable should be given priority. To this end, an efficient recommendation system should be considered to suggest and help tourists and travellers making a good decision. 
For aforementioned justifications, this project will investigate some common places that are highly recommended and visited in London. Then it can give reasonable suggestions for people to find the right place to enjoy their vacations or holidays.

## *2. Methodology* ##


### 2.1 Data ###

Many of the world’s largest tech companies rely on Foursquare data to add location into their apps and services. As the knowledge being acquired in this course, this project will take benefits from the FoursquareAPI application to access available data for the investigation. Foursquare lets users search for restaurants, nightlife spots, shops and other places of interest in their surrounding area. It is also possible to search other areas by entering the name of a remote location. The app displays personalized recommendations based on the time of day, displaying breakfast places in the morning, dinner places in the evening etc. Recommendations are personalized based on factors that include a user's check-in history, their "Tastes" and their venue ratings.
For this reason, this project can get information of London such as Postal codes and theirs Borough, the geographical locations including Latitude and Longitude, the common places that are offen visited and recommended by custumers like Hotels, Bars, Restaurants. This approach is easy because of the open-source of data base location given by Wikipedia. In this case, data about locations of London can be easily fetched from this URL link: https://en.wikipedia.org/wiki/List_of_areas_of_London. After collecting necessary data, it should be preprocessed in the following steps.

### 2.1 Handling data ###

Data obtained will be wrangled in this step. First, BeautifulSoup library in Python is used to read the HTML content of data, then obtained data will be fed into dataframe to extract the most informative attributes. Some rich information such as Postal codes, Borough, and Latitude & Longitude should be retained because most of places are recommended based on these information. After that, matching physical locations with geographical locations will be done by applying FoursquareAPI. This step is performed by providing Client ID, Secret ID of the user and Version of FoursquareAPI application. Then, based on data analysis, the most common places will be recommended. Finally, a map of these locations should be displayed to facilitate the visualization for custumers. Details of these processed will be given in the full report of Week 2.  

## *3. Data Acquisition and Processing* ##


_________

In [1]:
import numpy as np 
import pandas as pd


In [2]:
import requests
from bs4 import BeautifulSoup
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wikipedia_page = requests.get(wikipedia_link, headers = headers)
wikipedia_page

<Response [200]>

In [3]:
soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody
rows = table.find_all('tr')
columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
df = pd.DataFrame(columns = columns)
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)

        df
df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [4]:
df = df.rename(index=str, columns = {'Location': 'Location', 'London\xa0borough': 'Borough', 'Post town': 'Post-town', 'Postcode\xa0district': 'Postcode', 'Dial\xa0code': 'Dial-code', 'OS grid ref': 'OSGridRef'})
df['Borough'] = df['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df0 = df.drop('Postcode', axis=1).join(df['Postcode'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postcode'))
df1 = df0[['Location', 'Borough', 'Postcode', 'Post-town']].reset_index(drop=True)
df1.head()


,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Angel,Islington,EC1,LONDON
4,Angel,Islington,N1,LONDON


In [5]:
df2 = df1[['Location', 'Borough', 'Postcode']].reset_index(drop=True)
df2.head()

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Acton,"Ealing, Hammersmith and Fulham",W3
2,Acton,"Ealing, Hammersmith and Fulham",W4
3,Angel,Islington,EC1
4,Angel,Islington,N1


In [6]:
!pip -q install geocoder
import geocoder

In [7]:
def get_latlng(arcgis_geocoder):
    
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [8]:
postal_codes = df2['Postcode']    
coordinates = [get_latlng(postal_code) for postal_code in postal_codes[0:50].tolist()]

In [9]:
df_london = df2[0:50] # Only taking the first 50 neighborhoods
df_london_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_london['Latitude'] = df_london_coordinates['Latitude']
df_london['Longitude'] = df_london_coordinates['Longitude']

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
df_london.head()

,Location,Borough,Postcode,Latitude,Longitude
0,Abbey Wood,"Bexley, Greenwich",SE2,51.49245,0.12127
1,Acton,"Ealing, Hammersmith and Fulham",W3,51.51324,-0.26746
2,Acton,"Ealing, Hammersmith and Fulham",W4,51.48944,-0.26194
3,Angel,Islington,EC1,51.52361,-0.09877
4,Angel,Islington,N1,51.53792,-0.09983


In [11]:
df_london.shape

(50, 5)

In [12]:
CLIENT_ID = 'W0NEEQAZIGOMPCSXQILJZ3AL2BJJ4TZBXDCIMW5H2LKKUWO5' 
CLIENT_SECRET = 'K2XR2A4PZVPJEPLCKGJLMNEADID3234NR4MRBME0QMR2OEGV' 
VERSION = '20200311'

In [13]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
london_venues = getNearbyVenues(names=df_london['Borough'],
                                   latitudes=df_london['Latitude'],
                                   longitudes=df_london['Longitude']
                                  )

Bexley,  Greenwich 
Ealing, Hammersmith and Fulham
Ealing, Hammersmith and Fulham
Islington
Islington
Brent
Barnet
Lambeth, Wandsworth
Islington
Barnet, Enfield
Barnet
Havering
Merton
Barnet
Barnet
Bexley
Bromley
Bromley
Croydon
Kingston upon Thames
Croydon
Westminster
Hillingdon
Hounslow
Havering
Bexley
Barking and Dagenham
Islington
Enfield
Barnet, Brent, Camden
Lewisham
Bexley
Bexley
Haringey
Croydon
Bromley
Bromley
Bromley
Tower Hamlets
Bromley
Havering
Newham
Barking and Dagenham
Barking and Dagenham
Hackney
Dartford
Islington
Southwark
Lewisham
Bromley


In [15]:
london_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bexley, Greenwich",51.49245,0.12127,Sainsbury's,51.492824,0.120724,Supermarket
1,"Bexley, Greenwich",51.49245,0.12127,Lesnes Abbey,51.489526,0.125839,Historic Site
2,"Bexley, Greenwich",51.49245,0.12127,Lidl,51.496152,0.118417,Supermarket
3,"Bexley, Greenwich",51.49245,0.12127,Abbey Wood Railway Station (ABW),51.490825,0.123432,Train Station
4,"Bexley, Greenwich",51.49245,0.12127,The Abbey Arms,51.490693,0.121182,Pub


In [16]:
london_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Barking and Dagenham,201,201,201,201,201,201
Barnet,91,91,91,91,91,91
"Barnet, Brent, Camden",4,4,4,4,4,4
"Barnet, Enfield",98,98,98,98,98,98
Bexley,301,301,301,301,301,301
"Bexley, Greenwich",7,7,7,7,7,7
Brent,7,7,7,7,7,7
Bromley,251,251,251,251,251,251
Croydon,100,100,100,100,100,100


In [17]:
df_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")
df_onehot.drop(['Arcade'],axis=1,inplace=True) 
df_onehot.insert(loc=0, column='Neighborhood', value=london_venues['Neighborhood'] )
df_onehot.shape

(2689, 194)

In [18]:
df_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Xinjiang Restaurant,Yoga Studio
0,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_cluster = df_onehot.groupby('Neighborhood').mean().reset_index()
df_cluster.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Xinjiang Restaurant,Yoga Studio
0,Barking and Dagenham,0.0,0.0,0.009950,0.000000,0.0,0.029851,0.019900,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.039801,0.0,0.0,0.0,0.0
1,Barnet,0.0,0.0,0.000000,0.010989,0.0,0.000000,0.000000,0.010989,0.0,...,0.032967,0.0,0.0,0.010989,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Barnet, Brent, Camden",0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,"Barnet, Enfield",0.0,0.0,0.010204,0.000000,0.0,0.030612,0.020408,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.040816,0.0,0.0,0.0,0.0
4,Bexley,0.0,0.0,0.009967,0.000000,0.0,0.029900,0.019934,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.039867,0.0,0.0,0.0,0.0


In [20]:
top = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(top):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
def most_appeared_venues(row, top):
    categories = row.iloc[1:]
    categories_new = categories.sort_values(ascending=False)
    
    return categories_new.index.values[0:top]
neighborhoods = pd.DataFrame(columns=columns)
neighborhoods['Neighborhood'] = df_cluster['Neighborhood']

for i in np.arange(df_cluster.shape[0]):
    neighborhoods.iloc[i, 1:] = most_appeared_venues(df_cluster.iloc[ind, :], top)

neighborhoods.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
1,Barnet,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
2,"Barnet, Brent, Camden",Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
3,"Barnet, Enfield",Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
4,Bexley,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place


In [21]:
from sklearn.cluster import KMeans
k = 5
df_clustering = df_cluster.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=k, random_state=None).fit(df_clustering)
kmeans.labels_[0:10]
neighborhoods.insert(0, 'Cluster Labels', kmeans.labels_)
london_join = df_london
london_join = london_join.join(neighborhoods.set_index('Neighborhood'), on='Borough')
london_join.head()

,Location,Borough,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Wood,"Bexley, Greenwich",SE2,51.49245,0.12127,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
1,Acton,"Ealing, Hammersmith and Fulham",W3,51.51324,-0.26746,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
2,Acton,"Ealing, Hammersmith and Fulham",W4,51.48944,-0.26194,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
3,Angel,Islington,EC1,51.52361,-0.09877,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
4,Angel,Islington,N1,51.53792,-0.09983,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place


In [85]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode('London, UK')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [86]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

___

### London Map ###

___

In [93]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(london_join['Latitude'], london_join['Longitude'], london_join['Borough'], london_join['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Location Clustering ### 

____

In [22]:
london_join[london_join['Cluster Labels']==0]

,Location,Borough,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Church End,Brent,NW10,51.53916,-0.25123,0,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place


In [23]:
london_join[london_join['Cluster Labels']==1]

,Location,Borough,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Cockfosters,"Barnet, Enfield",EN4,51.506420,-0.127210,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
11,Collier Row,Havering,RM5,51.506420,-0.127210,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
15,Colyers,Bexley,DA8,51.506420,-0.127210,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
16,Aperfield,Bromley,TN16,51.271728,0.074711,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
17,Coney Hall,Bromley,BR4,51.506420,-0.127210,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
18,Coombe,Croydon,CR0,51.384755,-0.051499,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
19,Coombe,Kingston upon Thames,KT2,51.506420,-0.127210,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
20,Coulsdon,Croydon,CR5,51.506420,-0.127210,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
21,Covent Garden,Westminster,WC2,51.516510,-0.119680,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
22,Cowley,Hillingdon,UB8,51.506420,-0.127210,1,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place


In [24]:
london_join[london_join['Cluster Labels']==2]

,Location,Borough,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Wood,"Bexley, Greenwich",SE2,51.49245,0.12127,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
1,Acton,"Ealing, Hammersmith and Fulham",W3,51.51324,-0.26746,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
2,Acton,"Ealing, Hammersmith and Fulham",W4,51.48944,-0.26194,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
3,Angel,Islington,EC1,51.52361,-0.09877,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
4,Angel,Islington,N1,51.53792,-0.09983,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
6,Church End,Barnet,N3,51.60104,-0.19401,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
7,Clapham,"Lambeth, Wandsworth",SW4,51.46095,-0.13922,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
8,Clerkenwell,Islington,EC1,51.52361,-0.09877,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
10,Colindale,Barnet,NW9,51.58486,-0.24881,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
12,Colliers Wood,Merton,SW19,51.42170,-0.20796,2,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place


In [25]:
london_join[london_join['Cluster Labels']==3]

,Location,Borough,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Cricklewood,"Barnet, Brent, Camden",NW2,51.56237,-0.22131,3,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place


In [26]:
london_join[london_join['Cluster Labels']==4]

,Location,Borough,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Crews Hill,Enfield,EN2,51.65366,-0.110614,4,Hotel,Monument / Landmark,Theater,Wine Bar,Plaza,Cocktail Bar,Garden,Pub,Art Gallery,Sandwich Place
